# Multi-threshold Neuron - Part 1

In this first notebook I will try to formulate the simple maths and pseudocode to implement the concept of a new type of neuron proposed last year in July.

I have not check if this idea already exist in a code implementation. I would imagine it does in some form but I prefer to have a nice time without peeking into the answer :).

![model proposal comparison](images/model_proposal.png)

## Concepts

There are some main concepts in the Deep Learning domain that you should be familiar with before procedding. If you are familiar with them skip this part.

**Artificial neuron**

<img align='left'src="images/neuron_comparison.png" width="300px">
A mathematical representation of a biological neuron. They are the corner stone of artificial neural networks and Deep Learning in general. The idea is that the artificial neuron receives input signals from other connected artificial neurons and via a non-linear transmission function emits a signal itself. This transmission function is commonly known as the activation function.

**Activation function**

<img align='left'src="images/relu.png" width="300px">
The current understanding of a neuron is that it will transmit some signal only if the sum from incoming signals from other neurons exceeds a threshold. For an artificial neuron this is done via an activation function. There are many of them, and they also add non-linearity to the artificial neural network. There are many [activation functions](https://en.wikipedia.org/wiki/Activation_function) but the Rectified linear unit (ReLu) is recently one of the most broadly used in the Deep Learning community, and it's the one I will use in this notebook. The strict mathematical definition of the function is:

$$R(z) = max(0, z) =
     \begin{cases}
       0 &\quad\text{for } z\leq0 \\
       z &\quad\text{for } z > 0
     \end{cases}$$
     
You can check it out in the the official [Tensorflow code](https://github.com/tensorflow/tensorflow/blob/48be6a56d5c49d019ca049f8c48b2df597594343/tensorflow/compiler/tf2xla/kernels/relu_op.cc#L37) or in the [Tensorflow playground](https://github.com/tensorflow/playground/blob/718a6c8f2f876d5450b105e269534ae58e70223d/nn.ts#L120). (as you can notice the derivative is not continuous on $z=0$, this actually caused a bit of hessitation to adopt the activation function due to the fear of "braking" back-propagation which is based on derivating all elements of a neural network. So actually the derivative has to be written explicitly check tensorflow source code.)


## Scientific background

S. Sardi *et al.* publish in July last year an experimental work in [Nature scientific reports](https://www.nature.com/srep/) which contradicts a century old assumption about how neurons work. The work was a combined effort between the Physics, Life Sciences and Neuroscience departments of Bar-Ilan University in Tel Aviv.

![article_title](images/article.png)

The proposed neurons models contain a transmission function which in Deep Learning we refer to as [activation functions](https://en.wikipedia.org/wiki/Activation_function). The authors specifically refer to this function as the neuronal equations.  Their study put to the test three different conceptual neuron models, which I will name: centralized-single-threshold-neuron, centralized-multi-threshold-neuron and decentralized-multi-threshold-neuron.

**Centralized-Single-Threshold-Neuron**

This is the current adopted computational description of neurons ([artificial neurons](https://en.wikipedia.org/wiki/Artificial_neuron)), and the corner stone of Deep Learning. "A neuron consists of a unique centralized excitable mechanism". The signal reaching the neuron consists of a linear sum of the incoming signals from all the dendrites connected to the neuron. If this sum reaches a threshold, a spike signal is propagated through the axiom to the other connected neurons.

The neuronal equation of this model is:

$$I = \Theta\Big(\sum_{i=1}^NW_i\cdot I_i - t\Big)$$

where
- $i$: identifies any connected neuron
- $N$: total number of connected neurons
- $W_i$: is the weight (strenght) associated to the connection with neuron $i$
- $I_i$: is the signal comming out of neuron $i$
- $t$: is the centralized single neuron threshold 
- $\Theta$: is the heavyside step function
- $I$: signal output from the neuron




**Centralized-multi-threshold-neuron**

As for the Centralized-Single-Threshold-Neuron there is a centralized threshold applied by an activation function. The difference is that this model assumess non-linearity signal transmission in each dendrite and a threshold unit for each dendrite. This model is the most complex of the three and its neuronal equation can be written as:

$$I=\Theta(s - t)$$
$$s = \sum_{i=1}^Nf_i(W_i\cdot I_i)\cdot\Theta(W_i\cdot I_i - t_i)$$

where
- $i$: identifies any connected neuron
- $N$: total number of connected neurons
- $W_i$: is the weight (strenght) associated to the connection with neuron $i$
- $I_i$: is the signal comming out of neuron $i$
- $t_i$: is the threshold value for each neuron $i$
- $\Theta$: is the heavyside step function
- $f_i$: non-linear transfer function for connection $i$
- $t$: is the centralized single neuron threshold 
- $I$: signal output from the neuron

**Decentralized-Multi-Threshold-Neuron**

In this model the centralized threshold applied by the centralized activation function is removed. The neuron can be independently excited by any singal coming from a dendrite given that this signal is above a threshold. In escence this model describes a multi-threshold neuron. The mathematical representation is much simpler than the centralized-multi-threshold-neuron and it reads:

$$I=\sum_{i=1}^N\Theta(W_i\cdot I_i - t_i)$$
(here the authors write an OR, so this equation is correct if they meant the logical OR and not XOR.)

where
- $i$: identifies any connected neuron
- $N$: total number of connected neurons
- $W_i$: is the weight (strenght) associated to the connection with neuron $i$
- $I_i$: is the signal comming out of neuron $i$
- $t_i$: is the threshold value for each neuron $i$
- $\Theta$: is the heavyside step function
- $I$: signal output from the neuron

**Study conclusion**

Based on their experiments the authors conclude that the **Decentralized-Multi-Threshold-Neuron** model explains the data. The authors mention that the main reason for adopting the Centralized-Single-Threshold-Neuron as the main model, is that technology did not allow for direct excitation of single neurons, which other model experiments require. Moreover they state that these results could have been discovered using tehcnology that existed since the 1980s.

## The challenge

My idea is to take the Decentralized-Multi-Threshold-Neuron model and try to write an Deep Learning implementation, that is a neural network which will consist of multi-threshold neurons. From the current tools available for Deep Learning Tensorflow is quite flexible and allows for writting user defined implementations, if for some reason the flexibility is not there I will have to write the neural network myself (I hope not, even though I will recommend doing this at least once to really understand how the pieces of a neural network fit togethter.).

### Approximations

I am a theoretical physicist and as such it's impossible for me not to look for the [spherical cow](https://en.wikipedia.org/wiki/Spherical_cow).

**Single threshold value**

The multi-threshold neuron model contains different threshold parameter values ($t_i$). Mathematically a threshold has the same effect if we take it as a constant and instead the input signal is moved up or down by the connecting weights parameters. Hence, the neuronal equation becomes:

$$I=\sum_{i=1}^N\Theta(W_i\cdot I_i - t)$$

(this is also happening in the current neural network implementations, since in reality there is no reason for different neurons to have the same threshold, nevertheless commonly a single activation function is used on all neurons.)

**ReLu activation function**

The authors mention the neuronal equation, here I will take a step forward and define what in neuroscience is a transmission function. I basically will replace the heavisyde step function ($\Theta$) with threshold $t$ by a Rectified Linear Unit ($\mathcal{R}$). I write the transmission function as:

$$I=\sum_{i=1}^N\mathcal{R}(W_i\cdot I_i)$$

In general any activation function could replace the heavyside step function.

**Zero bias**

Notice that the proposed model equation contains no bias terms. For simplicity I will keep all bias equal to zero. Hopefully as a second iteration of the code architecture I will be able to add them easily.

### Backpropagation

In order for my neural network to be trained I will probably need backpropagation at some point, this means that the derivative of whatever function I introduce is necessary. Lucky for me I'm not changing the activation function equation itself, so I just use the already derivative of the ReLu function in Tensorflow:

$$\frac{d}{dz}\mathcal{R}(z)=
     \begin{cases}
       0 &\quad\text{for } z\leq0 \\
       1 &\quad\text{for } z > 0
     \end{cases}$$
     
You can check it out in the the official [Tensorflow code](https://github.com/tensorflow/tensorflow/blob/48be6a56d5c49d019ca049f8c48b2df597594343/tensorflow/compiler/tf2xla/kernels/relu_op.cc#L63) or in the [Tensorflow playground](https://github.com/tensorflow/playground/blob/718a6c8f2f876d5450b105e269534ae58e70223d/nn.ts#L121).

### Tensor multiplication

What I'm really changing is the architecture of the neural network as seen in Figure 0,  the activation function is no longer applied on the sum of all the inputs from the connected neurons, but instead on each input arriving from a connected neuron. In a simple description the sum over the arriving signal is going from inside the activation function to outside of it:


$$\mathcal{R}\Big(\sum_{i=1}^NW_i\cdot I_i\Big) \rightarrow \sum_{i=1}^N\mathcal{R}(W_i\cdot I_i)$$

Do you see the implementation problem described by the equation above?

In the current model (left equation) the input to the activation function $\sum_iW_i\cdot I_i$ is exactly the dot product between vectors $(W_1, W_2,\dots,W_N)$ and $(I_1, I_2,\dots,I_N)$ and it's this fact which allows fast computation of input signals for many neurons via matrix multiplication. In the fully connected layer for example, the input signal for $j$ neurons comming from connected neurons $i$ is $\sum_iW_{ji}\cdot I_i$, which means we can calculate the input signal for all $j$ neurons with one simple matrix multiplication $W \cdot I$ (given that $I$ is written as a $N\times1$ matrix).

In the new model this is no longer possible. I think this will be the biggest challenge, but AFAIK Tensorflow should provide enough flexibility to write this architecture. The idea is to keep things in the tensorflow world as all the functions in the package can be added to calculations and gradients will be automatically available for backpropagation.

It is also interesting to think about if the fact that we can do fast matrix multiplications lead us to prefer the current central-threshold model, like the modified Maslow's hammer saying goes "if all you have is a hammer, everything looks like a nail".

## Mathematical Implications

## Pseudo-code

** Tensorflow `relu_layer` **

In [7]:
import tensorflow as tf

In [55]:
sess = tf.Session()

In [59]:
w = tf.constant(2, shape=(10,10))
x = tf.constant(2, shape=(1, 10))
b = tf.constant(0, shape=(10,))
I = tf.nn.relu_layer(x, w , b)

In [60]:
sess.run(I)

array([[40, 40, 40, 40, 40, 40, 40, 40, 40, 40]], dtype=int32)

**Tensorflow relu function**

In [70]:
x = tf.constant(10, shape=(10,))
I = tf.nn.relu(x)

In [71]:
sess.run(I)

array([10, 10, 10, 10, 10, 10, 10, 10, 10, 10], dtype=int32)